In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import os
import sys
from deepface import DeepFace
from playwright.sync_api import sync_playwright

Mon objectif : 

Informations Personnelles


Nom complet
Date de naissance
Lieu de naissance
Sexe
Nationalité
État civil
Nombre d'enfants et âges
Coordonnées
Adresse résidentielle
Numéros de téléphone
Adresse email
Profils de médias sociaux
Caractéristiques Physiques
Taille
Poids
Couleur des cheveux
Couleur des yeux
Caractéristiques distinctives (cicatrices, tatouages, etc.)
Santé
Groupe sanguin
Historique médical
Allergies
Médicaments actuels
Conditions de santé chroniques
Éducation et Profession
Niveau d'éducation
Historique scolaire/universitaire
Compétences et qualifications
Expérience professionnelle
Titre de poste actuel
Employeur actuel
Historique de salaire
Finances
Revenu
Historique de crédit
Détails de compte bancaire
Investissements
Biens (immobilier, véhicules, etc.)
Préférences et Habitudes
Habitudes alimentaires
Préférences de loisirs
Habitudes de voyage
Préférences de consommation (marques, produits, etc.)
Habitudes de sommeil
Numérique
Historique de navigation
Données de localisation
Historique d'achats en ligne
Préférences de médias sociaux
Historique de recherche
Légal et Administratif
Numéro de sécurité sociale
Passeport/Numéro de carte d'identité
Permis de conduire
Registre des infractions pénales
Historique de vote
Autres
Appartenance à des groupes ou associations
Orientation sexuelle
Croyances religieuses
Opinions politiques
Relations familiales et sociales

In [21]:
# Définition de la classe Person
class Person:
    def __init__(self, nom, prenom, age, sexe, taille, photo):
        self.nom = nom
        self.prenom = prenom
        self.age = age
        self.sexe = sexe
        self.taille = taille
        self.photo = photo
        self.emotion = ""
        self.race = ""  
        taille_m = taille / 100  # Convertir la taille en mètres
        self.poids = 22 * (taille_m ** 2)
        
    def __str__(self):
        return pd.DataFrame({'Nom': [self.nom], 'Prénom': [self.prenom], 'Age': [self.age], 'Sexe': [self.sexe], 'Taille': [self.taille], 'Photo': [self.photo], 'Emotion': [self.emotion], 'Race': [self.race], 'Poids': [self.poids]}).to_string(index=False)
    
        

Input data

In [22]:
Nom = "Serano"
Prenom = "Henri"
Age = 21
Sexe = "Homme"
Taille = 180 #En cm
photo = "data//IMG_20230729_140527.jpg"
Person_to_study = Person(Nom, Prenom, Age, Sexe, Taille, photo)
print(Person_to_study)

   Nom Prénom  Age  Sexe  Taille                         Photo Emotion Race  Poids
Serano  Henri   21 Homme     180 data//IMG_20230729_140527.jpg               71.28


Analyse photo

In [15]:
try:
    analysis = DeepFace.analyze(Person_to_study.photo, actions=['age', 'gender', 'race', 'emotion'])
    print("Résultats bruts de l'analyse :", analysis)  # Imprimez l'ensemble de l'objet d'analyse
    analysis = analysis[0]

    age_estime = analysis.get("age", "Non disponible")
    
    Sexe_dict = analysis.get("gender", "Non disponible")
    if Sexe_dict['Man'] > Sexe_dict['Woman']:
        Sexe_estime = "Homme"
    else:
        Sexe_estime = "Femme"
    Person_to_study.race = analysis.get("dominant_race", "Non disponible")
    Person_to_study.emotion = analysis.get("dominant_emotion", "Non disponible")

    

except Exception as e:
    print("Une erreur s'est produite lors de l'analyse de l'image :", e)

Action: emotion: 100%|██████████| 4/4 [00:00<00:00,  6.16it/s]

Résultats bruts de l'analyse : [{'age': 23, 'region': {'x': 1598, 'y': 693, 'w': 309, 'h': 309}, 'gender': {'Woman': 0.25984132662415504, 'Man': 99.74015951156616}, 'dominant_gender': 'Man', 'race': {'asian': 3.5017691235599315e-06, 'indian': 5.843090292282313e-06, 'black': 6.229691019812323e-08, 'white': 99.69903234254929, 'middle eastern': 0.17765383446617908, 'latino hispanic': 0.12330520825174948}, 'dominant_race': 'white', 'emotion': {'angry': 10.830638756675869, 'disgust': 5.629888630282255e-05, 'fear': 0.04747551590961773, 'happy': 0.2195010165719966, 'sad': 3.357923375615671, 'surprise': 0.0036633128910054918, 'neutral': 85.54073744526656}, 'dominant_emotion': 'neutral'}, {'age': 29, 'region': {'x': 2762, 'y': 1750, 'w': 66, 'h': 66}, 'gender': {'Woman': 3.304857015609741, 'Man': 96.69514298439026}, 'dominant_gender': 'Man', 'race': {'asian': 5.423040688037872, 'indian': 0.4438098520040512, 'black': 1.621096394956112, 'white': 17.856645584106445, 'middle eastern': 74.0494430065

Vérification input data


In [18]:
if np.abs(Age - age_estime) > 5:
    print("L'âge estimé est très différent de l'âge réel.")
if Sexe != Sexe_estime:
    print("Le sexe estimé est différent du sexe réel.")
else:
    print("Le sexe estimé est le même que le sexe réel et l'âge aussi au passage.")

Le sexe estimé est le même que le sexe réel et l'âge aussi au passage.


Prédiction facile

In [19]:
def estimer_poids(taille_cm, imc_cible=22):
    """
    Estime le poids en kilogrammes basé sur la taille en centimètres et un IMC cible.

    :param taille_cm: Taille en centimètres
    :param imc_cible: Indice de masse corporelle cible (par défaut 22)
    :return: Poids estimé en kilogrammes
    """
    taille_m = taille_cm / 100  # Convertir la taille en mètres
    poids_estime = imc_cible * (taille_m ** 2)
    return poids_estime

poids = estimer_poids(Taille)
poids

71.28

Variable de bases : Travail en France

In [ ]:
Taux_de_Chômage = 7,4
habitants = 67162154
salaries = 19770837

tranche_cible_pop = 20787631 #25-49 ans
tranche_cible_emploi = 22862357 #25-54 ans

Import des différents dataset

In [32]:
datacouple = pd.read_excel("data//ip1774.xls", sheet_name="Figure 1")
datacouple = datacouple.set_index(datacouple.columns[0])
datacouple = datacouple.dropna()
datacouple

,Couples de même sexe Femmes,Couples de sexe différent Femmes,Couples de même sexe Hommes,Couples de sexe différent Hommes,Couples de même sexe ayant le bac Femmes,Couples de sexe différent ayant le bac Femmes,Couples de même sexe ayant le bac Homme,Couples de sexe différent ayant le bac Homme,Couples de même sexe ayant un bac +3 Femme,Couples de sexe différent ayant un bac +3 Femme,Couples de même sexe ayant un bac +3 Homme,Couples de sexe différent ayant un bac +3 Homme,Couples de même sexe Femme ayant un enfant,Couples de sexe différent ayant un enfant
Age,,,,,,,,,,,,,,
De 18 à 19 ans,0.2,6.8,0.2,2.7,73,70,71,61,0,0,0,0,9,26
De 20 à 24 ans,0.7,28.4,0.6,16.7,74,74,76,64,35,36,43,28,17,52
De 25 à 29 ans,0.9,57.1,1.0,45.0,78,75,76,65,41,36,45,29,31,79
De 30 à 34 ans,0.8,70.3,1.1,64.7,78,75,76,65,38,34,42,27,43,90
De 35 à 44 ans,0.7,71.5,0.9,71.5,78,71,79,64,39,32,45,27,44,91
De 45 à 54 ans,0.5,66.4,0.8,69.9,67,55,67,48,32,22,37,21,26,70
De 55 à 64 ans,0.3,64.8,0.5,72.1,60,41,60,38,27,14,31,17,13,23
65 ans ou plus,0.1,47.9,0.2,74.8,49,27,49,31,28,9,27,15,6,7
Ensemble,0.4,57.2,0.6,63.1,71,53,70,47,33,22,38,21,25,50


In [34]:
personnevivantseul = pd.read_excel("data//demo-couple-pers-seul-log-age.xlsx")
personnevivantseul = personnevivantseul.set_index(personnevivantseul.columns[0])
personnevivantseul

,Femmes,Hommes,Ensemble
Age,,,
15 à 19 ans,7.30,5.90,6.60
20 à 39 ans,15.40,20.40,17.90
20 à 24 ans,22.60,21.00,21.80
25 à 39 ans,13.30,20.30,16.70
40 à 64 ans,17.00,19.60,18.30
40 à 54 ans,12.10,18.80,15.40
55 à 64 ans,24.40,21.00,22.80
65 ans ou plus,43.50,21.50,33.90
65 à 79 ans,35.40,19.80,28.20


In [35]:
basecouple = pd.read_excel("data//base-ic-couples-familles-menages-2020.xlsx", sheet_name="IRIS")  
basecouple

,IRIS,Région,Département,Unité urbaine,Commune ou ARM,Libellé commune ou ARM,TRIRIS,Grand quartier,Libellé de l'IRIS,Type d'IRIS,...,Pop mén Pers Réf Autre en 2020 (compl),Familles en 2020 (compl),Fam Couple avec enfant(s) en 2020 (compl),Fam Monoparentales en 2020 (compl),Fam Couple sans enfant en 2020 (compl),Fam 0 enfant moins 25 ans en 2020 (compl),Fam 1 enfant moins 25 ans en 2020 (compl),Fam 2 enfants moins 25 ans en 2020 (compl),Fam 3 enfants moins 25 ans en 2020 (compl),Fam 4 enfants ou plus moins 25 ans en 2020 (compl)
0,IRIS,REG,DEP,UU2020,COM,LIBCOM,TRIRIS,GRD_QUART,LIBIRIS,TYP_IRIS,...,C20_PMEN_CS8,C20_FAM,C20_COUPAENF,C20_FAMMONO,C20_COUPSENF,C20_NE24F0,C20_NE24F1,C20_NE24F2,C20_NE24F3,C20_NE24F4P
1,010010000,84,01,01000,01001,L'Abergement-Clémenciat,ZZZZZZ,0100100,L'Abergement-Clémenciat (commune non irisée),Z,...,9.217454,257.361108,123.624477,23.12718,110.609451,124.519177,65.609777,47.688253,14.591798,4.952104
2,010020000,84,01,01000,01002,L'Abergement-de-Varey,ZZZZZZ,0100200,L'Abergement-de-Varey (commune non irisée),Z,...,0,76.390674,40.14897,4.784281,31.457423,31.457423,14.352843,25.157904,5.422504,0
3,010040101,84,01,01303,01004,Ambérieu-en-Bugey,ZZZZZZ,0100401,Les Pérouses-Triangle d'Activités,H,...,160.24815,358.137077,140.154206,54.008858,163.974014,169.714426,80.332698,66.261859,25.129763,16.698332
4,010040102,84,01,01303,01004,Ambérieu-en-Bugey,ZZZZZZ,0100401,Longeray-Gare,H,...,191.666204,926.880403,355.971473,199.683177,371.225752,429.778632,223.369136,148.568325,94.285942,30.878367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49269,974240101,04,974,9D201,97424,Cilaos,ZZZZZZ,9742401,La Ville-Les Thermes,H,...,245.192191,483.71223,198.657555,155.808238,129.246437,202.125511,119.868803,119.663574,18.964431,23.08991
49270,974240102,04,974,9D201,97424,Cilaos,ZZZZZZ,9742401,La Ville-Mare Sèche,H,...,346.038115,461.119157,204.249013,155.601768,101.268376,184.6641,133.533097,104.349619,38.572341,0
49271,974240103,04,974,9D201,97424,Cilaos,ZZZZZZ,9742401,Bras Sec,H,...,86.811077,204.609346,83.704368,66.997937,53.907041,79.836021,61.778865,49.35122,4.955601,8.68764
49272,974240104,04,974,9D201,97424,Cilaos,ZZZZZZ,9742401,Peter Both-Palmiste Rouge,H,...,123.183976,285.991022,162.393063,64.27678,59.321179,85.923189,109.753251,62.67989,14.223323,13.411368


In [36]:
print(basecouple.columns)
print(basecouple.shape)

Index(['IRIS', 'Région', 'Département', 'Unité urbaine', 'Commune ou ARM',
       'Libellé commune ou ARM', 'TRIRIS', 'Grand quartier',
       'Libellé de l'IRIS', 'Type d'IRIS', 'Label de l'IRIS',
       'Ménages en 2020 (compl)', 'Ménages 1 personne en 2020 (compl)',
       'Ménages Hommes seuls en 2020 (compl)',
       'Ménages Femmes seules en 2020 (compl)',
       'Ménages Autres sans famille en 2020 (compl)',
       'Ménages avec famille(s) en 2020 (compl)',
       'Mén fam princ Couple sans enfant en 2020 (compl)',
       'Mén fam princ Couple avec enfant(s) en 2020 (compl)',
       'Mén fam princ Famille mono en 2020 (compl)',
       'Pop Ménages en 2020 (compl)',
       'Pop mén Personnes seules en 2020 (compl)',
       'Pop mén Hommes seuls en 2020 (compl)',
       'Pop mén Femmes seules en 2020 (compl)',
       'Pop mén Autres sans famille en 2020 (compl)',
       'Pop mén avec famille(s) en 2020 (compl)',
       'Pop mén fam princ Couple sans enfant en 2020 (compl)',
      

In [39]:

def main():
    with sync_playwright() as p:
        try:
            browser = p.chromium.launch(headless=False)

            
            page = browser.new_page()
            page.goto("https://www.linkedin.com/uas/login?session_redirect=https%3A%2F%2Fwww%2Elinkedin%2Ecom%2Fsearch%2Fresults%2Fall%2F%3FfetchDeterministicClustersOnly%3Dtrue%26heroEntityKey%3Durn%253Ali%253Aorganization%253A1586%26keywords%3Damazon%26origin%3DRICH_QUERY_TYPEAHEAD_HISTORY%26position%3D0%26searchId%3Dafc6a59f-24b2-4236-8152-8fa35a3fe166%26sid%3DVPh&fromSignIn=true&trk=cold_join_sign_in", wait_until='networkidle')
           
            email = "TON MAIL AUSSI"
            mdp = "HEHE METS TON MDP ENCULER"
            prenom = "Bill" #Ouais je me fais pas chier
            nom = "Gates"
            
            email_field = page.wait_for_selector("id=username")
            email_field = page.query_selector("id=username")
            email_field.fill(email)
            password_field = page.wait_for_selector("id=password")
            password_field = page.query_selector("id=password")   
            password_field.fill(mdp)
            password_field.press("Enter")
            try:
                    page.goto("https://www.linkedin.com/search/results/people/?keywords="+str(prenom)+" "+str(nom)+"&origin=SWITCH_SEARCH_VERTICAL&sid=mqt", wait_until='networkidle')
                    
                    profil_link_selector = "a.app-aware-link[href*='linkedin.com/in']"
                    profil_link_element = page.query_selector(profil_link_selector)
                    
                    if profil_link_element:
                        profil_href = profil_link_element.get_attribute("href")
                        print(f"Le lien du profil est : {profil_href}")
                        # Naviguer vers le profil
                        page.goto(profil_href, wait_until='networkidle')
                    #page.goto("https://www.linkedin.com/in/williamhgates/", wait_until='networkidle')
                        
                    try:
                        
                        localisation = page.inner_text("span.text-body-small.inline.t-black--light.break-words")
                        print(f"Localisation: {localisation}")
                    except:
                        print("Problem with "+str(prenom)+" "+str(nom))
                    try:
                        titre = page.inner_text("div.JLTOxdAyPIbwAVfujRhErCSfXZcrNbXpHcBIE")
                        print(f"Titre: {titre}")
                    except:
                        print("Problem with "+str(prenom)+" "+str(nom))
                    try:
                        education = page.inner_text("section.pvs-list")   
                        print(education)
                        print(f"Education: {education}")
                    except:
                        print("Problem with "+str(prenom)+" "+str(nom))
                        

            except:
                    print("Problem with "+str(prenom)+" "+str(nom))
                    pass
                

                
            
        except Exception as e:
                print(f"An error occurred while scraping: {e}")



main()
    


Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.